In [3]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options
# Built-in modules
import math

# Imports from Qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution

# Imports from Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [2]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute

# Define the coin operators with specific theta values
def coin_operator(theta):
    return np.array([[np.cos(theta), np.sin(theta)],
                     [np.sin(theta), -np.cos(theta)]])

# Define parameters
n = 5  # Number of nodes on the cycle (odd number)
message = "01001110111001001011"

# Define theta values for different bit strings
theta1 = np.pi / 4
theta2 = np.pi / 8
theta3 = np.pi / 3
theta4 = np.pi / 6

# Create a quantum circuit
qc = QuantumCircuit(n, n)

# Apply coin operators based on the message
for i in range(len(message) // 2):
    message_slice = message[i * 2:i * 2 + 2]  # Get the next two bits
    theta = None
    if message_slice == "00":
        theta = theta1
    elif message_slice == "01":
        theta = theta2
    elif message_slice == "10":
        theta = theta3
    elif message_slice == "11":
        theta = theta4
    coin_op = coin_operator(theta)
    for j in range(n):
        # Apply the coin operator to the j-th node
        qc.unitary(coin_op, [j], label=message_slice)

# Measure the quantum circuit to obtain the probability distribution
qc.measure(range(n), range(n))

theta = np.pi / 4  # You can choose an appropriate initial θ value
for j in range(n):
    qc.u(theta, 0, 0, [j])

# Simulate the quantum circuit
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1000)
result = job.result()
counts = result.get_counts(qc)

# Calculate the desired length of the hash value
desired_length = n * 13

# Generate the hash value from the probability distribution
hash_value = ""
for key in counts:
    binary_string = key * (counts[key] * 104)  # Multiply by 104 as mentioned
    # Ensure that the binary strings are padded to the desired length (13)
    padded_binary_string = binary_string.zfill(13)
    hash_value += padded_binary_string

# Trim or extend the hash value to the desired length
if len(hash_value) < desired_length:
    hash_value = hash_value.ljust(desired_length, "0")
elif len(hash_value) > desired_length:
    hash_value = hash_value[:desired_length]

# Return the hash value as a binary string
print("Hash Value (Binary):", hash_value)

Hash Value (Binary): 00010000100001000010000100001000010000100001000010000100001000010


In [4]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute

# Define the coin operators with specific theta values
def coin_operator(theta):
    return np.array([[np.cos(theta), np.sin(theta)],
                     [np.sin(theta), -np.cos(theta)]])

# Define parameters
n = 5  # Number of nodes on the cycle (odd number)

# Define theta values for different bit strings
theta1 = np.pi / 4
theta2 = np.pi / 8
theta3 = np.pi / 3
theta4 = np.pi / 6

# Define the original message
original_message = "01001110111001001011"

# Define the five conditions
conditions = [
    original_message,
    original_message.replace("0", "1", 1),  # Change a 0 to 1
    original_message.replace("1", "0", 1),  # Change a 1 to 0
    original_message[1:],  # Delete the first bit
    original_message[:4] + "1" + original_message[4:],  # Insert a bit randomly
]

# Calculate the desired length of the hash value
desired_length = n * 13

for i, condition_message in enumerate(conditions, start=1):
    # Create a quantum circuit for the condition
    qc = QuantumCircuit(n, n)

    # Apply the initial state |ψ⟩ = |0⟩ cos(θ) + |1⟩ sin(θ)
    theta = np.pi / 4  # You can choose an appropriate initial θ value
    for j in range(n):
        qc.u(theta, 0, 0, [j])  # Apply a u-gate with θ as the angle

    # Apply coin operators based on the condition message
    for j in range(len(condition_message) // 2):
        message_slice = condition_message[j * 2:j * 2 + 2]  # Get the next two bits
        theta = None
        if message_slice == "00":
            theta = theta1
        elif message_slice == "01":
            theta = theta2
        elif message_slice == "10":
            theta = theta3
        elif message_slice == "11":
            theta = theta4
        coin_op = coin_operator(theta)
        for k in range(n):
            # Apply the coin operator to the k-th node
            qc.unitary(coin_op, [k], label=message_slice)

    # Measure the quantum circuit to obtain the probability distribution
    qc.measure(range(n), range(n))

    # Simulate the quantum circuit
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=1000)
    result = job.result()
    counts = result.get_counts(qc)

    # Generate the hash value from the probability distribution
    hash_value = ""
    for key in counts:
        binary_string = key * (counts[key] * 104)  # Multiply by 104 as mentioned
        # Ensure that the binary strings are padded to the desired length (13)
        padded_binary_string = binary_string.zfill(13)
        hash_value += padded_binary_string

    # Trim or extend the hash value to the desired length
    if len(hash_value) < desired_length:
        hash_value = hash_value.ljust(desired_length, "0")
    elif len(hash_value) > desired_length:
        hash_value = hash_value[:desired_length]

    # Print the hash value in binary format for the condition
    print(f"Condition {i} (Binary): {hash_value}")


Condition 1 (Binary): 10011100111001110011100111001110011100111001110011100111001110011
Condition 2 (Binary): 11111111111111111111111111111111111111111111111111111111111111111
Condition 3 (Binary): 01110011100111001110011100111001110011100111001110011100111001110
Condition 4 (Binary): 00010000100001000010000100001000010000100001000010000100001000010
Condition 5 (Binary): 00011000110001100011000110001100011000110001100011000110001100011


In [8]:
import numpy as np
import random
from qiskit import QuantumCircuit, Aer, execute

# Define the coin operators with specific theta values
def coin_operator(theta):
    return np.array([[np.cos(theta), np.sin(theta)],
                     [np.sin(theta), -np.cos(theta)]])

# Define parameters for hash generation
n = 5  # Number of nodes on the cycle (odd number)

# Define theta values for different bit strings
theta1 = np.pi / 4
theta2 = np.pi / 8
theta3 = np.pi / 3
theta4 = np.pi / 6

# Define a function to calculate the hash value for a message
def calculate_hash(message):
    # Create a quantum circuit for hash generation
    qc = QuantumCircuit(n, n)

    # Apply coin operators based on the message
    for i in range(len(message) // 2):
        message_slice = message[i * 2:i * 2 + 2]  # Get the next two bits
        theta = None
        if message_slice == "00":
            theta = theta1
        elif message_slice == "01":
            theta = theta2
        elif message_slice == "10":
            theta = theta3
        elif message_slice == "11":
            theta = theta4
        coin_op = coin_operator(theta)
        for j in range(n):
            # Apply the coin operator to the j-th node
            qc.unitary(coin_op, [j], label=message_slice)

    # Measure the quantum circuit to obtain the probability distribution
    qc.measure(range(n), range(n))

    # Choose an appropriate initial θ value
    theta = np.pi / 4
    for j in range(n):
        qc.u(theta, 0, 0, [j])

    # Simulate the quantum circuit
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=1000)
    result = job.result()
    counts = result.get_counts(qc)

    # Calculate the desired length of the hash value
    desired_length = n * 13

    # Generate the hash value from the probability distribution
    hash_value = ""
    for key in counts:
        binary_string = key * (counts[key] * 104)  # Multiply by 104 as mentioned
        # Ensure that the binary strings are padded to the desired length (13)
        padded_binary_string = binary_string.zfill(13)
        hash_value += padded_binary_string

    # Trim or extend the hash value to the desired length
    if len(hash_value) < desired_length:
        hash_value = hash_value.ljust(desired_length, "0")
    elif len(hash_value) > desired_length:
        hash_value = hash_value[:desired_length]

    return hash_value

# Define parameters for statistical analysis
N = 1024  # Number of tests

# Initialize variables for statistical analysis
B_mean = 0  # Mean normally changed bit number
A_mean = 0  # Mean added bit number of zero
T_mean = 0  # Mean sum of the changed bit number
P_mean = 0  # Mean changed probability
T_variance = 0  # Standard variance of the total changed bit number
P_variance = 0  # Standard variance of the changed probability

# Perform diffusion and confusion tests N times
for _ in range(N):
    # Step 1: Select a message randomly
    original_message = ''.join(random.choice("01") for _ in range(n))
    hash_original = calculate_hash(original_message)

    # Step 2: Change a bit of the original message randomly
    i = random.randint(0, n - 1)
    modified_message = original_message[:i] + ('1' if original_message[i] == '0' else '0') + original_message[i+1:]
    hash_modified = calculate_hash(modified_message)

    # Step 3: Compare the two hash values and count the changed bits
    changed_bits = sum(1 for bit1, bit2 in zip(hash_original, hash_modified) if bit1 != bit2)

    # Step 4: Count the added zero bits
    added_zeros = 13 - len(hash_original)

    # Update the mean values
    B_mean += changed_bits
    A_mean += added_zeros
    T_mean += (changed_bits + added_zeros)
    P_mean += (changed_bits / 221) * 100

# Calculate mean values
B_mean /= N
A_mean /= N
T_mean /= N
P_mean /= N

# Calculate standard variances
T_variance = sum(((changed_bits + added_zeros - T_mean) ** 2) for _ in range(N)) / (N - 1)
P_variance = sum((((changed_bits / 221) * 100 - P_mean) ** 2) for _ in range(N)) / (N - 1)

# Print the results
print("Mean normally changed bit number (B_mean):", B_mean)
print("Mean added bit number of zero (A_mean):", A_mean)
print("Mean sum of the changed bit number (T_mean):", T_mean)
print("Mean changed probability (P_mean):", P_mean)
print("Standard variance of the total changed bit number (T_variance):", T_variance)
print("Standard variance of the changed probability (P_variance):", P_variance)


Mean normally changed bit number (B_mean): 31.5859375
Mean added bit number of zero (A_mean): -52.0
Mean sum of the changed bit number (T_mean): -20.4140625
Mean changed probability (P_mean): 14.292279411764893
Standard variance of the total changed bit number (T_variance): 55.02205522971652
Standard variance of the changed probability (P_variance): 11.265546411766948


In [13]:
import random

# Define parameters for collision checking
N = 10000  # Number of tests
collisions_0 = 0
collisions_1 = 0

# Define the coin operators with specific theta values
def coin_operator(theta):
    return np.array([[np.cos(theta), np.sin(theta)],
                     [np.sin(theta), -np.cos(theta)]])

# Define parameters for hash generation
n = 5  # Number of nodes on the cycle (odd number)

# Define theta values for different bit strings
theta1 = np.pi / 4
theta2 = np.pi / 8
theta3 = np.pi / 3
theta4 = np.pi / 6

# Define a function to calculate the hash value for a message
def generate_hash(message):
    # Create a quantum circuit for hash generation
    qc = QuantumCircuit(n, n)

    # Apply coin operators based on the message
    for i in range(len(message) // 2):
        message_slice = message[i * 2:i * 2 + 2]  # Get the next two bits
        theta = None
        if message_slice == "00":
            theta = theta1
        elif message_slice == "01":
            theta = theta2
        elif message_slice == "10":
            theta = theta3
        elif message_slice == "11":
            theta = theta4
        coin_op = coin_operator(theta)
        for j in range(n):
            # Apply the coin operator to the j-th node
            qc.unitary(coin_op, [j], label=message_slice)

    # Measure the quantum circuit to obtain the probability distribution
    qc.measure(range(n), range(n))

    # Choose an appropriate initial θ value
    theta = np.pi / 4
    for j in range(n):
        qc.u(theta, 0, 0, [j])

    # Simulate the quantum circuit
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=1000)
    result = job.result()
    counts = result.get_counts(qc)

    # Calculate the desired length of the hash value
    desired_length = n * 13

    # Generate the hash value from the probability distribution
    hash_value = ""
    for key in counts:
        binary_string = key * (counts[key] * 104)  # Multiply by 104 as mentioned
        # Ensure that the binary strings are padded to the desired length (13)
        padded_binary_string = binary_string.zfill(13)
        hash_value += padded_binary_string

    # Trim or extend the hash value to the desired length
    if len(hash_value) < desired_length:
        hash_value = hash_value.ljust(desired_length, "0")
    elif len(hash_value) > desired_length:
        hash_value = hash_value[:desired_length]

    return hash_value

# Perform collision checks N times
for _ in range(N):
    # Step 1: Select a message randomly and generate the corresponding ASCII hash value
    original_message = ''.join(random.choice("01") for _ in range(n))
    ascii_hash_original = generate_hash(original_message)

    # Step 2: Insert a bit into the message randomly and generate the corresponding ASCII hash value
    i = random.randint(0, n - 1)
    modified_message = original_message[:i] + ('1' if original_message[i] == '0' else '0') + original_message[i+1:]
    ascii_hash_modified = generate_hash(modified_message)

    # Step 3: Compare the two ASCII hash values and count the number of matching characters
    matching_characters = sum(1 for c1, c2 in zip(ascii_hash_original, ascii_hash_modified) if c1 == c2)

    # Step 4: Update collision counts
    if matching_characters == 0:
        collisions_0 += 1
    elif matching_characters == 1:
        collisions_1 += 1

value = (collisions_0 - collisions_1) / N

# Print the results
print("Number of times = 0:", collisions_0)
print("Number of times = 1:", collisions_1)
print("Value =", value)



Number of times = 0: 153
Number of times = 1: 0
Value = 0.0153
